In [2]:
import tensorflow as tf
import numpy as np 
import matplotlib.pyplot as plt
from tensorflow.keras.layers import Input, Conv2D, Dense, Flatten, Dropout, GlobalMaxPooling2D,MaxPooling2D, BatchNormalization
from tensorflow.keras.models import Model 

In [3]:
cifar10 = tf.keras.datasets.cifar10
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_train, x_test = x_train/255.0, x_test/255.0
y_train, y_test = y_train.flatten(), y_test.flatten()
print(x_train.shape)
print(y_train.shape)

(50000, 32, 32, 3)
(50000,)


In [4]:
K = len(set(y_train))
print("number of classes", K)

number of classes 10


In [5]:
strategy = tf.distribute.MirroredStrategy()


INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:CPU:0',)


In [6]:
print(f'Number of Devices: {strategy.num_replicas_in_sync}')


Number of Devices: 1


In [7]:
def create_model():
    i = Input(shape=x_train[0].shape)
    x = Conv2D(32, (3, 3), strides=2, activation='relu')(i)
    x = Conv2D(64, (3, 3), strides=2, activation='relu')(x)
    x = Conv2D(128, (3, 3), strides=2, activation='relu')(x)
    x = Flatten()(x)
    x = Dropout(0.5)(x)
    x = Dense(1024, activation='relu')(x)
    x = Dropout(0.2)(x)
    x = Dense(K, activation='softmax')(x)
    
    model = Model(i, x)
    # Your model creation code here
    return model

In [8]:
strategy = tf.distribute.MirroredStrategy()

with strategy.scope():
  # Create your model using `create_model()` function
  model = create_model()

  # Compile the model
  model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])


INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:CPU:0',)


In [9]:
  r = model.fit(x_train, y_train, validation_data=(x_test, y_test),batch_size =128, epochs=15)


Epoch 1/15
391/391 ━━━━━━━━━━━━━━━━━━━━ 5s 10ms/step - accuracy: 0.3095 - loss: 1.8663 - val_accuracy: 0.5163 - val_loss: 1.3450
Epoch 2/15
391/391 ━━━━━━━━━━━━━━━━━━━━ 4s 10ms/step - accuracy: 0.5002 - loss: 1.3797 - val_accuracy: 0.5693 - val_loss: 1.2025
Epoch 3/15
391/391 ━━━━━━━━━━━━━━━━━━━━ 4s 10ms/step - accuracy: 0.5583 - loss: 1.2276 - val_accuracy: 0.5916 - val_loss: 1.1262
Epoch 4/15
391/391 ━━━━━━━━━━━━━━━━━━━━ 4s 10ms/step - accuracy: 0.5987 - loss: 1.1277 - val_accuracy: 0.6305 - val_loss: 1.0497
Epoch 5/15
391/391 ━━━━━━━━━━━━━━━━━━━━ 4s 10ms/step - accuracy: 0.6248 - loss: 1.0503 - val_accuracy: 0.6535 - val_loss: 0.9719
Epoch 6/15
391/391 ━━━━━━━━━━━━━━━━━━━━ 4s 10ms/step - accuracy: 0.6529 - loss: 0.9746 - val_accuracy: 0.6733 - val_loss: 0.9245
Epoch 7/15
391/391 ━━━━━━━━━━━━━━━━━━━━ 4s 10ms/step - accuracy: 0.6733 - loss: 0.9172 - val_accuracy: 0.6815 - val_loss: 0.9050
Epoch 8/15
391/391 ━━━━━━━━━━━━━━━━━━━━ 4s 10ms/step - accuracy: 0.6937 - loss: 0.8603 - val_acc

In [10]:
model2 = create_model()

In [11]:
model2.compile(optimizer='adam',
             loss ='sparse_categorical_crossentropy',
             metrics =['accuracy'])

r = model2.fit(x_train, y_train, validation_data =(x_test, y_test),batch_size=128, epochs =5)

Epoch 1/5
391/391 ━━━━━━━━━━━━━━━━━━━━ 5s 10ms/step - accuracy: 0.3255 - loss: 1.8385 - val_accuracy: 0.5226 - val_loss: 1.3359
Epoch 2/5
391/391 ━━━━━━━━━━━━━━━━━━━━ 4s 10ms/step - accuracy: 0.5141 - loss: 1.3619 - val_accuracy: 0.5743 - val_loss: 1.1969
Epoch 3/5
391/391 ━━━━━━━━━━━━━━━━━━━━ 4s 9ms/step - accuracy: 0.5656 - loss: 1.2168 - val_accuracy: 0.6013 - val_loss: 1.1119
Epoch 4/5
391/391 ━━━━━━━━━━━━━━━━━━━━ 4s 9ms/step - accuracy: 0.6067 - loss: 1.1066 - val_accuracy: 0.6335 - val_loss: 1.0264
Epoch 5/5
391/391 ━━━━━━━━━━━━━━━━━━━━ 4s 9ms/step - accuracy: 0.6312 - loss: 1.0254 - val_accuracy: 0.6622 - val_loss: 0.9613
